In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-185560
Azure region: southcentralus
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-185560


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "SCComClPrjNew" #"GIVE_A_CLUSTER_NAME"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found the one I created")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
    compute_target= ComputeTarget.create(ws,cluster_name,compute_config)
compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())



InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2022-02-14T23:24:11.123000+00:00', 'errors': None, 'creationTime': '2022-02-14T23:24:10.621142+00:00', 'modifiedTime': '2022-02-14T23:24:14.313973+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [15]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
#RandomParameterSampling(parameter_space, properties=None)
#{
#       "init_lr": uniform(0.0005, 0.005),
#       "hidden_size": choice(0, 100, 120, 140, 180)
#   }

### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
#"--learning_rate": normal(10,3),
#"--keep_probability": uniform(0.05,0.1),
#"--number_of_hidden_layers": choice(range(1,3)), 
"--P" : choice(0.01,0.1,1) ,     
"--batch_size" : choice(32,64,128)
}
)  

# Specify a Policy
#BanditPolicy(evaluation_interval=1, slack_factor=None, slack_amount=None, delay_evaluation=0)
policy =BanditPolicy(evaluation_interval=1, slack_factor=0.1, delay_evaluation=3) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
#SKLearn(source_directory, *, compute_target=None, vm_size=None, vm_priority=None, entry_script=None, script_params=None, use_docker=True, custom_docker_image=None, image_registry_details=None, user_managed=False, conda_packages=None, pip_packages=None, conda_dependencies_file_path=None, pip_requirements_file_path=None, conda_dependencies_file=None, pip_requirements_file=None, environment_variables=None, environment_definition=None, inputs=None, shm_size=None, resume_from=None, max_run_duration_seconds=None, framework_version=None, _enable_optimized_mode=False, _disable_validation=True, _show_lint_warnings=False, _show_package_warnings=False)
#src = './training' ### YOUR CODE HERE ###
skl_estimator = SKLearn(source_directory='./training',entry_script='train.py',compute_target=cluster_name)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
#HyperDriveConfig(hyperparameter_sampling, primary_metric_name, primary_metric_goal, max_total_runs, max_concurrent_runs=None, max_duration_minutes=10080, policy=None, estimator=None, run_config=None, resume_from=None, resume_child_runs=None, pipeline=None, debug_flag=None)
#PrimaryMetricGoal(value) MAXIMIZE, MINIMIZE 
hyperdrive_config = HyperDriveConfig(
        hyperparameter_sampling=ps,
        primary_metric_name='Accuracy',
        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
        max_total_runs = 20, 
        max_concurrent_runs =4,
        policy=policy,
        estimator=skl_estimator

) ### YOUR CODE HERE ###


In [16]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run=exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)
#print("end")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_031e19a8-f909-40a5-bfd4-5f607f207748
Web View: https://ml.azure.com/runs/HD_031e19a8-f909-40a5-bfd4-5f607f207748?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-185560/workspaces/quick-starts-ws-185560&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-02-14T23:10:20.780940][API][INFO]Experiment created<END>\n""<START>[2022-02-14T23:10:21.753908][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-02-14T23:10:23.244107][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"


ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

KeyError: 'log_files'

In [7]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics() 
parameter_values= best_run.get_details['runDefinotion']['Arguments']


print("Best Run ID :", best_run.id)
print("\n Accuracy :", best_run_metrics['accuracy'])
print("\n learning rate :", parameter_values[3])
print("\n keep probablity ", parametner_values[5])
print("\n batch size ", parametner_values[7])

print("\n best run file names :",best_run.get_file_names())
print("\n batch details ", best_run.get_details())

#print("\n best run details :",run.get_details())
print("\n best run metrics :",best_run_metrics)






AttributeError: 'NoneType' object has no attribute 'get_details'

In [ ]:
model = best_run.register_model(model_name='hyperdrive_best_run', 
model_path='outputs/model.joblib', tags={"Method" : "HyperDrive"},
properties={"Accuracy": best_run_metrics['accuracy']})

#model = best_run.register_model(model_name='hyperdrive_best_run', 
#model_path='outputs/model.joblib', tags={"Method" : "HyperDrive"},
#properties={"Accuracy": best_run.get_metrics()["Accuracy"]})

In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])


In [8]:
from train import clean_data
import pandas as pd
from sklearn.model_selection import train_test_split
from azureml.core import Dataset

# Use the clean_data function to clean your data.
x, y = clean_data(ds)


In [9]:
concat_x_y = pd.concat([x,y], axis=1)
TRAIN_d, TEST_d = train_test_split(concat_x_y, test_size=0.1, random_state=24)
TRAIN_d.to_csv('training/TRAIN_CSV.csv')


In [10]:

ds_new = ws.get_default_datastore()


In [11]:
ds_new.upload(src_dir='training',target_path='TRAIN_DATA')



Uploading an estimated of 3 files
Target already exists. Skipping upload for TRAIN_DATA/.amlignore
Target already exists. Skipping upload for TRAIN_DATA/.amlignore.amltmp
Target already exists. Skipping upload for TRAIN_DATA/TRAIN_CSV.csv
Uploaded 0 files


$AZUREML_DATAREFERENCE_7ac40f3ed9a144329bbcb76b5bd1ec48

In [13]:
TRAIN_DATASET = Dataset.Tabular.from_delimited_files(path=[(ds_new, ('TRAIN_DATA/TRAIN_CSV.csv'))])

In [14]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=TRAIN_DATASET,
    label_column_name='y',
    n_cross_validations=4)

In [15]:
# Submit your automl run
#helpp from https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train

### YOUR CODE HERE ###
remote_run = exp.submit(config= automl_config, show_output= True)



No run_configuration provided, running on local with default configuration
Running in the active local environment.


2022-02-14:23:33:34,348 INFO     [modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-02-14:23:33:34,367 INFO     [modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-02-14:23:33:53,630 INFO     [utils.py:159] NumExpr defaulting to 4 threads.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_4a4f105c-a3bc-4d62-b748-05b45d55bca0,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

In [ ]:
# Retrieve and save your best automl model.
# help from https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train

### YOUR CODE HERE ###
automl_best_run, automl_fitted_model = remote_run.get_output()

test_run = next(automl_best_run.get_children(type='automl.model_test'))
test_run.wait_for_completion(show_output=False, wait_post_processing=True)

# Get test metrics
test_run_metrics = test_run.get_metrics()
for name, value in test_run_metrics.items():
    print(f"{name}: {value}")

# Get test predictions as a Dataset
test_run_details = test_run.get_details()
dataset_id = test_run_details['outputDatasets'][0]['identifier']['savedId']
test_run_predictions = Dataset.get_by_id(workspace, dataset_id)
predictions_df = test_run_predictions.to_pandas_dataframe()

# Alternatively, the test predictions can be retrieved via the run outputs.
test_run.download_file("predictions/predictions.csv")
predictions_df = pd.read_csv("predictions.csv")





print(hasattr(automl_fitted_model, 'steps'))
print("\n")
print(automl_best_run.get_metrics())
print("\n")
print(automl_best_run.get_tags())

In [ ]:
# Save your best automl model.
#save_onnx_model(onnx_model: onnx.onnx_ml_pb2.ModelProto, file_path: str) -> None 
#help from https://docs.microsoft.com/en-us/python/api/azureml-automl-runtime/azureml.automl.runtime.onnx_convert.onnx_converter.onnxconverter?view=azure-ml-py
#help from https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#configure-your-experiment-settings


from azureml.automl.runtime.onnx_convert import OnnxConverter

automl_best_run, automl_fitted_model = remote_run.get_output(return_onnx_model=True)

OnnxConverter.save_onnx_model(automl_fitted_model, './outputs/Automl_Model.onnx')

In [ ]:
compute_target.delete()